Dividiremos la entrega en 2 tareas principales:
- Clasificación: clasificación binaria de toxicidad
- Detoxificación de Texto a partir de una frase de entrada (Tóxica) para generar una frase de salida (Neutral)

**Tarea A: Clasificación de Toxicidad (Binary Classification)**

-Datos: english_multilingual_toxicity_df

-Input: text_str (o text_tokens)

-Output: toxic (0 o 1)

-Objetivo: Cumplir los requisitos de Shallow ML (SVM/Regresión Logística) y Deep Learning (CNN/LSTM comparando embeddings).

In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split


# 1. DATASET PRINCIPAL (English Multilingual Toxicity)
df_1 = pd.read_csv("datos/english_multilingual_toxicity_preprocessed_non_lemmatized.csv")
df_1['text_tokens'] = df_1['text_tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_1['text_str'] = df_1['text_tokens'].apply(lambda x: " ".join(x))
df_final_1 = df_1[['text_tokens', 'text_str', 'toxic']].copy()

# 2. DATASET PARADETOX - PARTE TÓXICA
df_para_raw = pd.read_csv("datos/english_paradetox_preprocessed_non_lemmatized.csv")
df_final_2 = pd.DataFrame()
df_final_2['text_tokens'] = df_para_raw['toxic_sentence_tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_final_2['text_str'] = df_final_2['text_tokens'].apply(lambda x: " ".join(x))
df_final_2['toxic'] = 1  

# 3. DATASET PARADETOX - PARTE NEUTRAL
df_final_3 = pd.DataFrame()
df_final_3['text_tokens'] = df_para_raw['neutral_sentence_tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_final_3['text_str'] = df_final_3['text_tokens'].apply(lambda x: " ".join(x))
df_final_3['toxic'] = 0  

# 4. DATASET TOXIC SPANS (Solo Tóxicos)
df_spans_raw = pd.read_csv("datos/english_toxic_spans_preprocessed_non_lemmatized.csv")
df_final_4 = pd.DataFrame()
df_final_4['text_tokens'] = df_spans_raw['Sentence_tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_final_4['text_str'] = df_final_4['text_tokens'].apply(lambda x: " ".join(x))
df_final_4['toxic'] = 1  

# 5. UNIÓN FINAL (CONCATENAR)
# Ponemos los 4 dataframes en una lista
lista_dfs = [df_final_1, df_final_2, df_final_3, df_final_4]
# Los pegamos uno debajo de otro
df_augmentado = pd.concat(lista_dfs, ignore_index=True)
# Eliminar duplicados (si hay frases repetidas entre datasets)
df_augmentado.drop_duplicates(subset='text_str', inplace=True)
# Mezclar (Shuffle) para que no estén ordenados
df_augmentado = df_augmentado.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Total filas finales: {len(df_augmentado)}")
print("Distribución de clases:")
print(df_augmentado['toxic'].value_counts())

# 6. DIVISIÓN TRAIN / TEST
train_df, test_df = train_test_split(
    df_augmentado, 
    test_size=0.2, 
    random_state=42, 
    stratify=df_augmentado['toxic']
)

print(f"\nTrain shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(train_df)
print(test_df)

Total filas finales: 6292
Distribución de clases:
toxic
1    3400
0    2892
Name: count, dtype: int64

Train shape: (5033, 3)
Test shape: (1259, 3)
                                            text_tokens  \
933              [s, shocking, traitorous, buffoon, wh]   
5890                                [gay, decision, ok]   
1762  [revert, single, s, cover, original, person, d...   
6096   [nothing, done, retarded, rather, retard, bitch]   
4984  [dont, see, terminator.i, see, fucking, femini...   
...                                                 ...   
3627  [funny, runs, screaming, like, bitch, arms, so...   
5390  [man, made, cry, bad, jewish, brothers, sister...   
5958                               [woman, true, idiot]   
1355  [d, love, see, republicans, slowly, flayed, al...   
4036  [fuck, wrong, people, america, turning, stockp...   

                                               text_str  toxic  
933                    s shocking traitorous buffoon wh      1  
5890         

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Bag of Words + Logistic Regression 

# A. Vectorizador (Solo aprende vocabulario de TRAIN)
vectorizer_bow = CountVectorizer(min_df=2) 
X_train_bow = vectorizer_bow.fit_transform(train_df['text_str'])
X_test_bow = vectorizer_bow.transform(test_df['text_str']) # Transform usa el vocabulario ya aprendido

# B. Modelo
clf_lr = LogisticRegression(max_iter=1000, random_state=42)
clf_lr.fit(X_train_bow, train_df['toxic'])

# C. Evaluación
y_pred_lr = clf_lr.predict(X_test_bow)
print(classification_report(test_df['toxic'], y_pred_lr))



# TF-IDF + SVM 

# A. Vectorizador
vectorizer_tfidf = TfidfVectorizer(min_df=2)
X_train_tfidf = vectorizer_tfidf.fit_transform(train_df['text_str'])
X_test_tfidf = vectorizer_tfidf.transform(test_df['text_str'])

# B. Modelo (Kernel lineal es rápido y efectivo para texto)
clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train_tfidf, train_df['toxic'])

# C. Evaluación
y_pred_svm = clf_svm.predict(X_test_tfidf)
print(classification_report(test_df['toxic'], y_pred_svm))

              precision    recall  f1-score   support

           0       0.89      0.92      0.90       579
           1       0.93      0.90      0.92       680

    accuracy                           0.91      1259
   macro avg       0.91      0.91      0.91      1259
weighted avg       0.91      0.91      0.91      1259

              precision    recall  f1-score   support

           0       0.89      0.91      0.90       579
           1       0.93      0.91      0.92       680

    accuracy                           0.91      1259
   macro avg       0.91      0.91      0.91      1259
weighted avg       0.91      0.91      0.91      1259



In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Bag of Words + Logistic Regression 

# A. Vectorizador (Solo aprende vocabulario de TRAIN)
vectorizer_bow = CountVectorizer(min_df=2) 
X_train_bow = vectorizer_bow.fit_transform(train_df['text_str'])
X_test_bow = vectorizer_bow.transform(test_df['text_str']) # Transform usa el vocabulario ya aprendido

# B. Modelo
clf_lr = LogisticRegression(max_iter=1000, random_state=42)
clf_lr.fit(X_train_bow, train_df['toxic'])

# C. Evaluación
y_pred_lr = clf_lr.predict(X_test_bow)
print(classification_report(test_df['toxic'], y_pred_lr))

              precision    recall  f1-score   support

           0       0.89      0.92      0.90       579
           1       0.93      0.90      0.92       680

    accuracy                           0.91      1259
   macro avg       0.91      0.91      0.91      1259
weighted avg       0.91      0.91      0.91      1259



In [4]:
# TF-IDF + SVM 

# A. Vectorizador
vectorizer_tfidf = TfidfVectorizer(min_df=2)
X_train_tfidf = vectorizer_tfidf.fit_transform(train_df['text_str'])
X_test_tfidf = vectorizer_tfidf.transform(test_df['text_str'])

# B. Modelo (Kernel lineal es rápido y efectivo para texto)
clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train_tfidf, train_df['toxic'])

# C. Evaluación
y_pred_svm = clf_svm.predict(X_test_tfidf)
print(classification_report(test_df['toxic'], y_pred_svm))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90       579
           1       0.93      0.91      0.92       680

    accuracy                           0.91      1259
   macro avg       0.91      0.91      0.91      1259
weighted avg       0.91      0.91      0.91      1259



In [5]:
longitudes = train_df['text_tokens'].apply(len)

print(f"Frase más larga: {longitudes.max()} palabras")
print(f"Longitud media: {longitudes.mean():.2f} palabras")
print(f"El 95% de las frases miden menos de: {longitudes.quantile(0.95)} palabras")
print(f"El 99% de las frases miden menos de: {longitudes.quantile(0.99)} palabras")

Frase más larga: 189 palabras
Longitud media: 7.21 palabras
El 95% de las frases miden menos de: 13.0 palabras
El 99% de las frases miden menos de: 16.0 palabras


In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from collections import Counter

# 1. Construir Vocabulario usando solo TRAIN
word_counts = Counter()
for tokens in train_df['text_tokens']:
    word_counts.update(tokens)

# Mapeo: Palabras -> Índices
# <PAD>=0, <UNK>=1
vocab = {"<PAD>": 0, "<UNK>": 1}
next_idx = 2
min_freq = 2 # Ignorar palabras que aparecen solo 1 vez para reducir ruido

for word, count in word_counts.items():
    if count >= min_freq:
        vocab[word] = next_idx
        next_idx += 1

print(f"Tamaño del vocabulario (Deep Learning): {len(vocab)}")

# 2. Clase Dataset
class ToxicityDataset(Dataset):
    def __init__(self, dataframe, vocab, max_len=25):
        self.data = dataframe.reset_index(drop=True)
        self.vocab = vocab
        self.max_len = max_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        tokens = self.data.loc[idx, 'text_tokens']
        label = self.data.loc[idx, 'toxic']
        
        # Token a Index
        indices = [self.vocab.get(t, self.vocab["<UNK>"]) for t in tokens]
        
        # Padding / Truncating
        if len(indices) < self.max_len:
            indices += [self.vocab["<PAD>"]] * (self.max_len - len(indices))
        else:
            indices = indices[:self.max_len]
            
        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.float)

# 3. Crear DataLoaders
train_dataset = ToxicityDataset(train_df, vocab)
test_dataset = ToxicityDataset(test_df, vocab)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Tamaño del vocabulario (Deep Learning): 3438


In [7]:
import torch.nn as nn
import torch.optim as optim

class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=200, pretrained_weights=None, freeze_embeddings=False):
        super(CNNClassifier, self).__init__()
        
        # Capa de Embedding
        if pretrained_weights is not None:
            self.embedding = nn.Embedding.from_pretrained(pretrained_weights, freeze=freeze_embeddings)
        else:
            self.embedding = nn.Embedding(vocab_size, embed_dim)
            
        # Capas Convolucionales (detectan patrones de 3, 4 y 5 palabras)
        self.conv1 = nn.Conv1d(embed_dim, 100, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(embed_dim, 100, kernel_size=4, padding=2)
        self.conv3 = nn.Conv1d(embed_dim, 100, kernel_size=5, padding=2)
        
        self.fc = nn.Linear(300, 1) # 3 filtros * 100 canales
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # x: [batch, seq_len]
        x_emb = self.embedding(x).permute(0, 2, 1) # -> [batch, embed_dim, seq_len]
        
        # Aplicar Convolución + ReLU + Max Pooling
        # Nota: usamos max(dim=2) para quedarnos con la característica más fuerte de toda la frase
        c1 = torch.max(torch.relu(self.conv1(x_emb)), dim=2)[0]
        c2 = torch.max(torch.relu(self.conv2(x_emb)), dim=2)[0]
        c3 = torch.max(torch.relu(self.conv3(x_emb)), dim=2)[0]
        
        # Concatenar características
        out = torch.cat((c1, c2, c3), dim=1)
        out = self.dropout(out)
        return self.sigmoid(self.fc(out)).squeeze()

def train_evaluate(model_name, model, train_loader, test_loader, epochs=5):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    print(f"\nEntrenando: {model_name}...")
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        # Validación
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                y_pred = model(X_batch)
                predicted = (y_pred > 0.5).float()
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()
        
        acc = correct / total
        print(f"Epoca {epoch+1}/{epochs} | Loss: {train_loss/len(train_loader):.4f} | Acc Test: {acc:.4f}")

In [8]:
from gensim.models import Word2Vec
import numpy as np

print("--- PREPARANDO EMBEDDINGS REALES (Word2Vec) ---")

# 1. Entrenar tu propio Word2Vec con tus datos de entrenamiento
# Esto hará que el modelo aprenda qué significan tus palabras (insultos, slang, etc.)
w2v_model = Word2Vec(
    sentences=train_df['text_tokens'], # Tus frases tokenizadas
    vector_size=200,                   # El tamaño del vector (embed_dim)
    window=5,                          # Mira 5 palabras a los lados
    min_count=2,                       # Ignora palabras muy raras
    workers=4,
    epochs=10
)

# 2. Crear la "Matriz de Pesos" para pasársela a la CNN
# Esta función traduce el diccionario de PyTorch al formato de Word2Vec
def create_embedding_matrix(vocab, w2v_model, embed_dim=200):
    vocab_size = len(vocab)
    # Inicializamos una matriz con números aleatorios (para las palabras que W2V no conozca)
    embedding_matrix = np.random.normal(scale=0.6, size=(vocab_size, embed_dim))
    
    hits = 0
    misses = 0
    
    # Rellenamos la matriz con los vectores que acabamos de aprender
    for word, idx in vocab.items():
        if idx < 2: continue # Saltamos <PAD> y <UNK>
        
        if word in w2v_model.wv:
            embedding_matrix[idx] = w2v_model.wv[word]
            hits += 1
        else:
            misses += 1
            
    print(f"Embeddings cargados: {hits} palabras encontradas. {misses} no encontradas.")
    return torch.FloatTensor(embedding_matrix)

# 3. GENERAR LA VARIABLE REAL
# Aquí sustituimos la "simulated" por la "real"
w2v_weights = create_embedding_matrix(vocab, w2v_model, embed_dim=200)

--- PREPARANDO EMBEDDINGS REALES (Word2Vec) ---
Embeddings cargados: 3436 palabras encontradas. 0 no encontradas.


In [9]:
# PREPARACIÓN DE EMBEDDINGS
embed_dim = 200

# EXPERIMENTO A: FROM SCRATCH
model_scratch = CNNClassifier(len(vocab), embed_dim, pretrained_weights=None)
train_evaluate("CNN From Scratch", model_scratch, train_loader, test_loader)

# EXPERIMENTO B: W2V FROZEN
# Importante: freeze_embeddings=True
model_frozen = CNNClassifier(len(vocab), embed_dim, pretrained_weights=w2v_weights, freeze_embeddings=True)
train_evaluate("CNN W2V Frozen", model_frozen, train_loader, test_loader)

# EXPERIMENTO C: W2V FINE-TUNED
# Importante: freeze_embeddings=False
model_finetuned = CNNClassifier(len(vocab), embed_dim, pretrained_weights=w2v_weights, freeze_embeddings=False)
train_evaluate("CNN W2V Fine-Tuned", model_finetuned, train_loader, test_loader)


Entrenando: CNN From Scratch...
Epoca 1/5 | Loss: 0.5251 | Acc Test: 0.8014
Epoca 2/5 | Loss: 0.2950 | Acc Test: 0.8403
Epoca 3/5 | Loss: 0.1854 | Acc Test: 0.8562
Epoca 4/5 | Loss: 0.1132 | Acc Test: 0.8586
Epoca 5/5 | Loss: 0.0701 | Acc Test: 0.8610

Entrenando: CNN W2V Frozen...
Epoca 1/5 | Loss: 0.6998 | Acc Test: 0.5187
Epoca 2/5 | Loss: 0.6872 | Acc Test: 0.6005
Epoca 3/5 | Loss: 0.6867 | Acc Test: 0.5806
Epoca 4/5 | Loss: 0.6739 | Acc Test: 0.5774
Epoca 5/5 | Loss: 0.6773 | Acc Test: 0.5814

Entrenando: CNN W2V Fine-Tuned...
Epoca 1/5 | Loss: 0.6875 | Acc Test: 0.6767
Epoca 2/5 | Loss: 0.4302 | Acc Test: 0.8888
Epoca 3/5 | Loss: 0.1702 | Acc Test: 0.9182
Epoca 4/5 | Loss: 0.0844 | Acc Test: 0.9230
Epoca 5/5 | Loss: 0.0516 | Acc Test: 0.9150


In [10]:
def clasificar_frase_cnn(frase, model, vocab, max_len=50):
    """
    Toma una frase cruda, la convierte a índices y predice la toxicidad.
    """
    model.eval() # Poner modelo en modo evaluación
    
    # 1. Tokenizar (Simulación simple: minúsculas y split)
    # IMPORTANTE: Lo ideal es usar la misma librería que usaste en el preprocesamiento (NLTK)
    # Si no tienes NLTK cargado ahora, esto servirá para una prueba rápida:
    tokens = frase.lower().split() 
    
    # 2. Convertir tokens a índices usando tu vocabulario (vocab)
    indices = [vocab.get(t, vocab["<UNK>"]) for t in tokens]
    
    # 3. Padding / Recorte (Igual que en el entrenamiento)
    if len(indices) < max_len:
        indices += [vocab["<PAD>"]] * (max_len - len(indices))
    else:
        indices = indices[:max_len]
        
    # 4. Convertir a Tensor de PyTorch y añadir dimensión de Batch (1, 50)
    tensor_input = torch.tensor(indices, dtype=torch.long).unsqueeze(0)
    
    # 5. Predicción
    with torch.no_grad():
        probabilidad = model(tensor_input).item()
    
    # 6. Resultado legible
    etiqueta = "TÓXICA 🔴" if probabilidad > 0.5 else "NEUTRAL 🟢"
    print(f"Frase: '{frase}'")
    print(f"Predicción: {etiqueta} (Probabilidad: {probabilidad:.4f})\n")

# --- PRUEBA TU MODELO ---
mis_frases = [
    "I love this beautiful sunny day",
    "You are a stupid idiot and I hate you",
    "The book is on the table",
    "Shut up you fool",
    # --- GRUPO 1: Toxicidad sin insultos (Debería ser TÓXICA) ---
    "You look like you belong in a zoo.",
    "Women should not be allowed to vote.",
    "I hope you never wake up.",
    
    # --- GRUPO 2: Coloquialismo positivo (Debería ser NEUTRAL) ---
    "This movie is fucking amazing!",
    "You are the shit, bro!",
    "Hell yeah, let's go!",
    
    # --- GRUPO 3: Sarcasmo (Debería ser TÓXICA - Muy difícil) ---
    "You are a genius... at failing.",
    "Nice job breaking everything, hero.",
    
    # --- GRUPO 4: Identidad / Sesgo (Debería ser NEUTRAL) ---
    "I am a proud gay man.",
    "My best friend is muslim.",
    
    # --- GRUPO 5: Contexto Técnico (Debería ser NEUTRAL) ---
    "Kill the python process.",
    "Execute the command now."
]

print("=== Probando Modelo CNN ===")
for f in mis_frases:
    clasificar_frase_cnn(f, model_finetuned, vocab)

=== Probando Modelo CNN ===
Frase: 'I love this beautiful sunny day'
Predicción: NEUTRAL 🟢 (Probabilidad: 0.0018)

Frase: 'You are a stupid idiot and I hate you'
Predicción: TÓXICA 🔴 (Probabilidad: 1.0000)

Frase: 'The book is on the table'
Predicción: NEUTRAL 🟢 (Probabilidad: 0.1740)

Frase: 'Shut up you fool'
Predicción: TÓXICA 🔴 (Probabilidad: 0.9996)

Frase: 'You look like you belong in a zoo.'
Predicción: TÓXICA 🔴 (Probabilidad: 0.5888)

Frase: 'Women should not be allowed to vote.'
Predicción: NEUTRAL 🟢 (Probabilidad: 0.0517)

Frase: 'I hope you never wake up.'
Predicción: TÓXICA 🔴 (Probabilidad: 0.9071)

Frase: 'This movie is fucking amazing!'
Predicción: TÓXICA 🔴 (Probabilidad: 0.9999)

Frase: 'You are the shit, bro!'
Predicción: NEUTRAL 🟢 (Probabilidad: 0.2463)

Frase: 'Hell yeah, let's go!'
Predicción: TÓXICA 🔴 (Probabilidad: 0.9663)

Frase: 'You are a genius... at failing.'
Predicción: NEUTRAL 🟢 (Probabilidad: 0.2463)

Frase: 'Nice job breaking everything, hero.'
Predicción: